## Web scrapping using python

#### References
1. [Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)
2. [Web Scraping using Python](https://www.datacamp.com/community/tutorials/web-scraping-using-python)

In [ ]:
# $ python3 -m venv venv
# $ . ./venv/bin/activate

In [8]:
#Better
!pip install requests BeautifulSoup4 fire

  Using cached https://files.pythonhosted.org/packages/34/a7/0e22e70778aca01a52b9c899d9c145c6396d7b613719cd63db97ffa13f2f/fire-0.3.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/8a/48/a76be51647d0eb9f10e2a4511bf3ffb8cc1e6b14e9e4fab46173aa79f981/termcolor-1.1.0.tar.gz
  Stored in directory: /Users/yabebal/Library/Caches/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
  Stored in directory: /Users/yabebal/Library/Caches/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
Successfully built fire termcolor
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire

In [2]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

In [3]:
res = get_elements('https://africafreak.com/100-most-influential-twitter-users-in-africa', tag='h2')

In [4]:
inf_acc = res[-6::-1]
inf_acc
top_10 = inf_acc[:10]
top_10

['1. Trevor Noah (@Trevornoah)',
 '2. Gareth Cliff (@GarethCliff)',
 '3. Jacob G. Zuma (@SAPresident)',
 '4. News24 (@News24)',
 '5. Julius Sello Malema (@Julius_S_Malema)',
 '6. Helen Zille (@helenzille)',
 '7. mailandguardian (@mailandguardian)',
 '8. 5FM (@5FM)',
 '9. loyiso gola (@loyisogola)',
 '10. Computicket (@Computicket)']

In [5]:
names, twitter_ids = [], []
for acc in top_10:
    splitted = acc[3::].split("(")
    name,twitter_id = splitted
    name = name[:-1]
    twitter_id = twitter_id.strip(")")
    names.append(name)
    twitter_ids.append(twitter_id)
    
df = pd.DataFrame({'names':names,'twitter_ids':twitter_ids})
df.to_csv(r'C:\Users\Ajikobi Oluwasegun\Desktop\Data_Science\10 Academy\week 1\Monday\jupyter notebooks\top_10_influencers.csv',index=False)

In [6]:
url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'
response = simple_get(url)

In [7]:
res_gov = get_elements(response, search={'find_all':{'class_':'twitter-tweet'}})
res_gov
strings = []
for tag in res_gov:
  if tag.string != None:
    strings.append(tag.string)


findaing all of {'class_': 'twitter-tweet'}


In [9]:
new_strings = [x for x in strings if len(x) < 70]
gov_acc = []
for i in range(0,len(new_strings),2):
    gov_acc.append(new_strings[i])

In [11]:
ten_gov_acc = gov_acc[:10]
ten_gov_acc

['— Eswatini Government (@EswatiniGovern1) ',
 '— Malawi Government (@MalawiGovt) ',
 '— Hage G. Geingob (@hagegeingob) ',
 '— Seychelles Ministry of Finance (@FinanceSC) ',
 '— PresidencyZA (@PresidencyZA) ',
 '— Ministry of Health Zambia (@mohzambia) ',
 '— President of Zimbabwe (@edmnangagwa) ',
 '— MinSantédj (@MinSantedj) ',
 '— Yemane G. Meskel (@hawelti) ',
 '— State House Kenya (@StateHouseKenya) ']

In [12]:
names, twitter_ids = [], []

for acc in ten_gov_acc:
    splitted = acc.strip("—, ")[:-1].split(" (")
    name, twitter_id = splitted
    names.append(name)
    twitter_ids.append(twitter_id)
    
df1 = pd.DataFrame({'names':names,'twitter_ids':twitter_ids})
df1.to_csv(r'C:\Users\Ajikobi Oluwasegun\Desktop\Data_Science\10 Academy\week 1\Monday\jupyter notebooks\ten_gov_officials.csv',index=False)

## Web scrapping using bash script
If the web site has a quite simple HTML, you can easily use curl to perform the request and then extract the needed values using bash commands grep, cut , sed, ..

This tutorial is adapted from [this](https://medium.com/@LiliSousa/web-scraping-with-bash-690e4ee7f98d) medium article

In [98]:
%%bash 

# curl the page and save content to tmp_file
#url = "https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa"
#curl -X GET $url -o tmp_file


#!/bin/bash

# write headers to CSV file
echo "Name, twitter_id" >> extractData.csv
n="1"
while [ $n -lt 2 ]
do
  
  #get title
  title=$(cat tmp_file | grep "class=\"twitter-tweet\"" | cut -d ';' -f1 )
  echo $title
  #get author
  #twitter_id=$(cat tmp_file |grep -A1 "class=\"css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0\"" | tail -1)

  #echo "$title, $twitter_id" >> extractData.csv
  #echo "$title, $twitter_id"
    
  n=$[$n+1]

done

Couldn't find program: 'bash'
